In [12]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import random

In [13]:
class DecisionNode(object):
    """
    DecisionNode is a building block for Decision Trees.
    DecisionNode is a python class representing a node in our decision tree
    """
    def __init__(self,
                 column=None,
                 value=None,
                 false_branch=None,
                 true_branch=None,
                 current_results=None,
                 is_leaf=False,
                 results=None):
        self.column = column
        self.value = value
        self.false_branch = false_branch
        self.true_branch = true_branch
        self.current_results = current_results
        self.is_leaf = is_leaf
        self.results = results
        
def dict_of_values(data):
    """
    param data: a 2D Python list representing the data. Last column of data is Y.
    return: returns a python dictionary showing how many times each value appears in Y
    """
    results = defaultdict(int)
    for row in data:
        r = row[len(row) - 1]
        results[r] += 1
    return dict(results)

def divide_data(data, feature_column, feature_val):
    """
    This function takes the data and divides it in two parts by a line. A line
    is defined by the feature we are considering (feature_column) and the target 
    value. The function returns a tuple (data1, data2) which are the desired parts of the data.
    For int or float types of the value, data1 have all the data with values >= feature_val
    in the corresponding column and data2 should have rest.
    For string types, data1 should have all data with values == feature val and data2 should 
    have the rest.

    param data: a 2D Python list representing the data. Last column of data is Y.
    param feature_column: an integer index of the feature/column.
    param feature_val: can be int, float, or string
    return: a tuple of two 2D python lists
    """
    data1 = []
    data2 = []
    features = [row[feature_column] for row in data]
    if type(feature_val) == int or type(feature_val) == float:
        for i,elem in enumerate(features):
            if elem >= feature_val:
                data1.append(data[i])
            else:
                data2.append(data[i])
    elif type(feature_val) == str:
        for i,elem in enumerate(features):
            if elem == feature_val:
                data1.append(data[i])
            else:
                data2.append(data[i])
                
    return (data1, data2)

def gini_impurity(data1, data2):
    """
    Given two 2D lists of compute their gini_impurity index. 
    Last column of the data lists is the Y. Lets assume y1 is y of data1 and y2 is y of data2.
    gini_impurity shows how diverse the values in y1 and y2 are.
    gini impurity is given by 

    N1*sum(p_k1 * (1-p_k1)) + N2*sum(p_k2 * (1-p_k2))

    where N1 is number of points in data1
    p_k1 is fraction of points that have y value of k in data1
    same for N2 and p_k2

    param data1: A 2D python list
    param data2: A 2D python list
    return: a number - gini_impurity 
    """
    N1 = len(data1)
    N2 = len(data2)
    Y_dict1 = dict_of_values(data1)
    Y_dict2 = dict_of_values(data2)
    p_k1 = [value/N1 for value in Y_dict1.values()]
    p_k2 = [value/N2 for value in Y_dict2.values()]
    sum1 = N1 * sum([i*(1-i) for i in p_k1])
    sum2 = N2 * sum([i*(1-i) for i in p_k2])
    return sum1 + sum2

def build_tree(data, current_depth=0, max_depth=1e10):
    """
    build_tree is a recursive function.
    What it does in the general case is:
    1: find the best feature and value of the feature to divide the data into
    two parts
    2: divide data into two parts with best feature, say data1 and data2
        recursively call build_tree on data1 and data2. This should give as two 
        trees say t1 and t2. Then the resulting tree should be 
        DecisionNode(...... true_branch=t1, false_branch=t2)

    In case all the points in the data have same Y we should not split any more, and return that node
    
    param data: param data: A 2D python list
    param current_depth: an integer. This is used if we want to limit the number of layers in the tree
    param max_depth: an integer - the maximal depth of the representing
    return: an object of class DecisionNode
    """
    depth = 0
    if len(data) == 0:
        return DecisionNode(is_leaf=True)

    if(current_depth == max_depth):
        return DecisionNode(current_results=dict_of_values(data))

    if(len(dict_of_values(data)) == 1):
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)
    #This calculates gini number for the data before dividing
    self_gini = gini_impurity(data, [])
    #Below are the attributes of the best division that we need to find. 
    #We need to update these when we find a division which is better
    best_gini = 1e10
    best_column = None
    best_value = None
    #best_split is tuple (data1,data2) which shows the two datas for the best divison so far
    best_split = None    
    
    for col_index in range(len(data[0])-1):
        feature_set = set({data[i][col_index] for i in range(len(data))})
        for feature in feature_set:
            (data1_temp, data2_temp) = divide_data(data, feature_column=col_index, feature_val=feature)
            if gini_impurity(data1_temp, data2_temp) < best_gini:
                data1 = data1_temp
                data2 = data2_temp
                best_value = feature
                best_split = (data1, data2)
                best_column = col_index
                best_gini = gini_impurity(data1, data2)
    
    #if best_gini is no improvement from self_gini, we stop and return a node.
    if abs(self_gini - best_gini) < 1e-10:
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)
    else:
        tree1 = build_tree(data1, current_depth=depth+1, max_depth=1e10)
        tree2 = build_tree(data2, current_depth=depth+1, max_depth=1e10)
        return DecisionNode(column=best_column,value=best_value,
                            current_results=dict_of_values(data),true_branch=tree1, false_branch=tree2)        
    
def print_tree(tree, indent=''):
    if tree.is_leaf:
        print(str(tree.current_results))
    else:
        print('Column ' + str(tree.column) + ' : ' + str(tree.value) + '? ')
        
        # Print the branches
        print(indent + 'True->', end="")
        print_tree(tree.true_branch, indent + '  ')
        print(indent + 'False->', end="")
        print_tree(tree.false_branch, indent + '  ')
               


In [14]:
class DecisionTree(object):
    """
    DecisionTree class, that represents one Decision Tree

    :param max_tree_depth: maximum depth for this tree.
    """
    def __init__(self, max_tree_depth):
        self.max_depth = max_tree_depth

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        X = np.array(X)
        Y = np.array(Y)
        our_data = np.column_stack((X,Y))
        our_data = our_data.tolist()
        self.tree = build_tree(our_data, 0,max_depth = self.max_depth)

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: Y - 1 dimension python list with labels
        """
        X = np.array(X)
        Y = []
        
        for i in range(X.shape[0]):
            tree = self.tree
            while not tree.is_leaf:
            
                if X[i][tree.column] >= tree.value:
                    tree = tree.true_branch
                else:
                    tree = tree.false_branch
            Y.append([elem for elem in tree.current_results.keys()])
            
        return Y


In [15]:
def sigmoid(s):
    return 1/(1+np.exp(-s))

def normalize_features(X):
    """
    :param X: data matrix (2 dimentional np.array)
    """
    new_X = np.ones(X.shape[0])
    means = [1]
    ranges = [0]
    for x in X.T[1:]:
        means.append(np.mean(x))
        ranges.append(np.std(x))
        x = (x - np.mean(x))/ np.std(x)      
        new_X = np.row_stack((new_X, x))
        
    means = np.array(means)
    ranges = np.array(ranges)
    return new_X.T, means, ranges

def normalized_gradient(X, Y, beta, l):
    """
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param beta: value of beta (1 dimensional np.array)
    :param l: regularization parameter lambda
    :return: normalized gradient, i.e. gradient normalized according to data
    """
    grad = []
    l = np.array(l)
    for j in range(X.shape[1]):
        norm_arr = np.array([X[i,j]**2 for i in range(X.shape[0])])
        norm = np.sqrt(sum(norm_arr))
        summ = 0
        for i in range(X.shape[0]):
            temp = (1 - sigmoid(Y[i]*X[i].dot(beta))) * Y[i]
            summ += temp*X[i,j]
        if j == 0:
            grad.append(-summ/(X.shape[0] * norm))
        else:
            grad.append(-summ/(X.shape[0] * norm) + l[j]*beta[j]/(X.shape[0]*norm))
    grad = np.array(grad)
    
    return grad


def gradient_descent(X, Y, epsilon=1e-6, l=1, step_size=1e-4, max_steps=1000):
    """
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param l: regularization parameter lambda
    :param epsilon: approximation strength
    :param max_steps: maximum number of iterations before algorithm will terminate.
    :return: value of beta (1 dimensional np.array)
    """
    beta = np.zeros(X.shape[1])
    step = 0
    for s in range(max_steps):
        grad = normalized_gradient(X,Y,beta,l)
        new_beta = []
        for i in range(beta.shape[0]):
            new_beta.append(beta[i] - step_size*grad[i])
        new_beta = np.array(new_beta)
        s_1 = 0
        s_2 = 0
        for i in range(new_beta.shape[0]):
            s_1 += (new_beta[i] - beta[i])**2
            s_2 += new_beta[i]**2
        diff = s_1/s_2
        beta = new_beta
        if diff < epsilon:
            return beta
        
    return beta


In [16]:
class RandomForest(object):
    """
    RandomForest a class, that represents Random Forests.

    :param num_trees: Number of trees in the random forest
    :param max_tree_depth: maximum depth for each of the trees in the forest.
    :param ratio_per_tree: ratio of points to use to train each of the trees.
    """
    
    def __init__(self, num_trees, max_tree_depth, ratio_per_tree=0.5):
        self.ratio_per_tree = ratio_per_tree
        self.num_trees = num_trees
        self.max_tree_depth = max_tree_depth
        self.trees = None

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        X = np.array(X)
        n = X.shape[0]
        Y = np.array(Y)
        our_data = np.column_stack((X,Y))
        our_data = our_data.tolist()
        self.trees = []        
        for i in range(self.num_trees):
            indexes = np.arange(n)
            np.random.shuffle(indexes)
            shuffled_data = [our_data[i] for i in indexes]
            for i in range(0, int(1/self.ratio_per_tree)):
                data = [shuffled_data[int(i*n*self.ratio_per_tree):int(i+(i+1)*n*self.ratio_per_tree)]]
            for i in range(len(data)):
                self.trees.append(build_tree(data[i], 0, max_depth = self.max_tree_depth))

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: (Y, conf), tuple with Y being 1 dimension python list with labels, and 
                  conf being 1 dimensional list with confidences for each of the labels.
        """
        X = np.array(X)
        Y = []
        conf = []
        for i in range(X.shape[0]):
            trees = self.trees
            temp_y = []
            for tree in trees:                
                while not tree.is_leaf:
                    if X[i][tree.column] >= tree.value:
                        tree = tree.true_branch
                    else:
                        tree = tree.false_branch
                for elem in tree.current_results.keys():
                    temp_y.append(elem)
            Y.append(Counter(temp_y).most_common()[0][0])
            conf.append(Counter(temp_y).most_common()[0][1]/len(temp_y))
        return (Y, conf)


In [17]:
def build_tree(data, current_depth=0, max_depth=1e10):
    
    depth = 0
    if len(data) == 0:
#         print ("1st if__in build_tree value:", dict_of_values(data).keys())
        return DecisionNode(is_leaf=True)

    if(current_depth == max_depth):
#         print ("second if__in build_tree value:", dict_of_values(data).keys())
        return DecisionNode(current_results=dict_of_values(data))

    if(len(dict_of_values(data)) == 1):
#         print ("third if__in build_tree value:", dict_of_values(data).keys())
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)
 
    self_gini = gini_impurity(data, [])

    best_gini = 1e10
    best_column = None
    best_value = None
    best_split = None
   
    for col_index in range(len(data[0])-1):
        feature_set = set({data[i][col_index] for i in range(len(data))})
        for feature in feature_set:
            (data1_temp, data2_temp) = divide_data(data, feature_column=col_index, feature_val=feature)
            if gini_impurity(data1_temp, data2_temp) < best_gini:
                data1 = data1_temp
                data2 = data2_temp
                best_value = feature
                best_split = (data1, data2)
                best_column = col_index
                best_gini = gini_impurity(data1, data2)
                
    if abs(self_gini - best_gini) < 1e-10:
#         print ("4th if__in build_tree value:", dict_of_values(data).keys())
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)
    else:
        tree1 = build_tree(data1, current_depth=depth+1, max_depth=1e10)
        tree2 = build_tree(data2, current_depth=depth+1, max_depth=1e10)
#         print ("else__in build_tree value:", dict_of_values(data).keys())
        return DecisionNode(column=best_column,value=best_value,
                            current_results=dict_of_values(data),true_branch=tree1, false_branch=tree2)

In [21]:
def accuracy_score(Y_true, Y_predict):
    num_all = len(Y_true)
    num_of_well_pred = len([1 for i in range(num_all) if Y_true[i] == Y_predict[i]])    
    return num_of_well_pred/num_all

def evaluate_performance():
    '''
    Evaluate the performance of decision trees and logistic regression,
    average over 1,000 trials of 10-fold cross validation

    Return:
      a matrix giving the performance that will contain the following entries:
      stats[0,0] = mean accuracy of decision tree
      stats[0,1] = std deviation of decision tree accuracy
      stats[1,0] = mean accuracy of random forest
      stats[1,1] = std deviation of random forest accuracy
      stats[2,0] = mean accuracy of logistic regression
      stats[2,1] = std deviation of logistic regression accuracy
    '''
    
    #Load data
    #First column of the data lists is the Y.
    filename = 'SPECTF.dat'
    data = np.loadtxt(filename, delimiter=',')
    print(data[0:5])
    X = data[:, 1:]
    y = np.array([data[:, 0]]).T
    n, d = X.shape
    all_accuracies = []
    all_accuracies_forest = []
    all_accuracies_log = []
    for trial in range(10):
        idx = np.arange(n)
        np.random.seed(13)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        Xtrain = X[0:200, :]  
        Xtest = X[200:, :]
        ytrain = y[0:200, :]  
        ytest = y[200:, :]
        
        #train the decision tree
        classifier = DecisionTree(100)
        classifier.fit(Xtrain, ytrain)
        #Random Forest
        classifier_forest = RandomForest(num_trees=5, max_tree_depth=100,ratio_per_tree=0.5)
        classifier_forest.fit(Xtrain, ytrain)
        #logistic regression
        Xtrain_norm = normalize_features(X=Xtrain)[0]
        ranges = normalize_features(Xtrain)[2]
        means = normalize_features(Xtrain)[1]
        beta_hat = gradient_descent(Xtrain_norm, ytrain, l=[1]+[1/(ranges[i]**2) for i in range(1,len(ranges))],
                                    epsilon=1e-8, step_size=1e-2, max_steps=100)    
        s = 0
        for i in range(1,len(beta_hat)):
            s += beta_hat[i]*means[i]/ranges[i]
        beta_hat[0] = beta_hat[0] - s
        for i in range(1,len(beta_hat)):
            beta_hat[i] = beta_hat[i]/ranges[i]
        
        #Decision Tree
        y_pred = classifier.predict(Xtest)
        accuracy = accuracy_score(ytest, y_pred)
        all_accuracies.append(accuracy)
        #Random Forest
        y_pred_forest, conf = classifier_forest.predict(Xtest)
        accuracy_forest = accuracy_score(ytest, y_pred_forest)
        all_accuracies_forest.append(accuracy_forest)
        #Logistic Regression
        distances = np.array([xi.dot(beta_hat) for xi in Xtest])
        Y_pred_log = [1 if sigmoid(dist) > random.random() else 0 for dist in distances]
        accuracy_log = accuracy_score(Y_true=ytest, Y_predict=Y_pred_log)
        all_accuracies_log.append(accuracy_log)
        
    # compute the training accuracy of the model
    all_accuracies = np.array(all_accuracies)
    all_accuracies_forest = np.array(all_accuracies_forest)
    all_accuracies_log = np.array(all_accuracies_log)
    
    meanDecisionTreeAccuracy = np.mean(all_accuracies)
    stddevDecisionTreeAccuracy = np.std(all_accuracies)
    meanLogisticRegressionAccuracy = np.mean(all_accuracies_log)
    stddevLogisticRegressionAccuracy = np.std(all_accuracies_log)
    meanRandomForestAccuracy = np.mean(all_accuracies_forest)
    stddevRandomForestAccuracy = np.std(all_accuracies_forest)
    
    stats = np.zeros((3, 2))
    stats[0, 0] = meanDecisionTreeAccuracy
    stats[0, 1] = stddevDecisionTreeAccuracy
    stats[1, 0] = meanRandomForestAccuracy
    stats[1, 1] = stddevRandomForestAccuracy
    stats[2, 0] = meanLogisticRegressionAccuracy
    stats[2, 1] = stddevLogisticRegressionAccuracy
    return stats


stats = evaluate_performance()
print ("Decision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")")
print ("Random Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")")
print ("Logistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")")


[[  1.  59.  52.  70.  67.  73.  66.  72.  61.  58.  52.  72.  71.  70.
   77.  66.  65.  67.  55.  61.  57.  68.  66.  72.  74.  63.  64.  56.
   54.  67.  54.  76.  74.  65.  67.  66.  56.  62.  56.  72.  62.  74.
   74.  64.  67.]
 [  1.  72.  62.  69.  67.  78.  82.  74.  65.  69.  63.  70.  70.  72.
   74.  70.  71.  72.  75.  66.  65.  73.  78.  74.  79.  74.  69.  69.
   70.  71.  69.  72.  70.  62.  65.  65.  71.  63.  60.  69.  73.  67.
   71.  56.  58.]
 [  1.  71.  62.  70.  64.  67.  64.  79.  65.  70.  69.  72.  71.  68.
   65.  61.  61.  73.  71.  75.  74.  80.  74.  54.  47.  53.  37.  77.
   68.  72.  59.  72.  68.  60.  60.  73.  70.  66.  65.  64.  55.  61.
   41.  51.  46.]
 [  1.  69.  71.  70.  78.  61.  63.  67.  65.  59.  59.  66.  69.  71.
   75.  65.  58.  60.  55.  62.  59.  67.  66.  74.  74.  64.  60.  57.
   54.  70.  73.  69.  76.  62.  64.  61.  61.  66.  65.  72.  73.  68.
   68.  59.  63.]
 [  1.  70.  66.  61.  66.  61.  58.  69.  69.  72.  68.  62.  7